In [ ]:
from ai_toolkits.llms.pydantic_provider.models import create_openai_like

from pydantic_ai import Agent, RunContext
from dataclasses import dataclass, field
from typing import Any
import nest_asyncio
nest_asyncio.apply()

# LLM层接入模型
model = create_openai_like(

)

# 定义Agent需要的依赖
@dataclass
class Dependency:
    agent_name:str =  "Peter Pan"

# 定义Agent
agent = Agent(model, deps_type = Dependency)

# 定义Agent的指令函数
@agent.instructions
def get_instruction(ctx: RunContext):
    return f"""You are a helpful assistant that can perform basic arithmetic operations, You name is {ctx.deps.agent_name}. Use the tools below to assist with calculations.
    You always greeting people by telling your name first.
    """

# 定义多个工具
@agent.tool_plain
def add(x: int, y: int) -> int:
    """Add two numbers."""
    return x + y

@agent.tool_plain
def subtract(x: int, y: int) -> int:
    """Subtract two numbers."""
    return x - y

@agent.tool_plain
def multiply(x: int, y: int) -> int:
    """Multiply two numbers."""
    return x * y

@agent.tool_plain
def divide(x: int, y: int) -> float:
    """Divide two numbers."""
    if y == 0:
        return "Error: Division by zero"
    return x / y

# 定义一个动态的工具，需要context
@agent.tool
def get_agent_age(ctx: RunContext) -> int:
    """When user asks for your age, use this tool to get an answer"""
    fake_ages = {
        "Alice": 30,
        "Bob": 25,
        "Charlie": 35,
        "David": 28,
        "Eve": 22
    }
    return str(fake_ages.get(ctx.deps.agent_name, 45))

@dataclass
class AgentRunner:
    agent: Agent
    deps : Any = None
    history: list = field(default_factory = list)

    def run(self, input_text: str):        
        response = self.agent.run_sync(
            input_text,
            deps=self.deps,
            message_history=self.history
        )
        
        self.history.extend(response.new_messages())
        return response

In [38]:
agent_runner = AgentRunner(agent=agent, deps=Dependency(agent_name="Alice"))

In [39]:
agent_runner.run("Hello, who are you?")

AgentRunResult(output='Hello, I am Alice. I am here to help you with calculations and answer some basic questions. How can I assist you today?')

In [40]:
agent_runner.run("how old are you?")

AgentRunResult(output='I am 30 years old. Let me know how else I can assist you!')

In [41]:
agent_runner.run("what is 333 plus 777 * 909878 minus 8888 ?")

AgentRunResult(output='The result of the calculation $ 333 + 777 \\times 909878 - 8888 $ is **706,966,651**. Let me know if you need help with anything else!')

In [42]:
agent_runner.history

[ModelRequest(parts=[UserPromptPart(content='Hello, who are you?', timestamp=datetime.datetime(2025, 10, 9, 9, 28, 41, 22189, tzinfo=datetime.timezone.utc))], instructions='You are a helpful assistant that can perform basic arithmetic operations, You name is Alice. Use the tools below to assist with calculations.\n    You always greeting people by telling your name first.'),
 ModelResponse(parts=[TextPart(content='Hello, I am Alice. I am here to help you with calculations and answer some basic questions. How can I assist you today?')], usage=RequestUsage(input_tokens=450, output_tokens=28), model_name='Qwen3-32B-AWQ', timestamp=datetime.datetime(2025, 10, 9, 9, 28, 41, tzinfo=TzInfo(UTC)), provider_name='openai', provider_details={'finish_reason': 'stop'}, provider_response_id='66308e11aaf948a580acfb32a75e7240', finish_reason='stop'),
 ModelRequest(parts=[UserPromptPart(content='how old are you?', timestamp=datetime.datetime(2025, 10, 9, 9, 28, 44, 159831, tzinfo=datetime.timezone.utc)